In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt


In [15]:
df = pd.read_csv('datosDelitos.csv',index_col=0, low_memory=False)
df["fecha"] = pd.to_datetime(df["fecha"], infer_datetime_format=True, format='mixed')
df["semana"] = df['fecha'].dt.strftime('%W').astype(int)
df.head(3)

,fecha,franja_horaria,barrio,delitos,id_comuna,anio,mes,dia_semana,horario,semana
0,2016-01-01,13,Parque Avellaneda,Robo,Comuna 9,2016,1,Friday,Tarde,0
1,2016-01-01,22,Palermo,Robo,Comuna 14,2016,1,Friday,Noche,0
2,2016-01-01,1,Parque Patricios,Lesiones por Siniestro Vial,Comuna 4,2016,1,Friday,Madrugada,0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637109 entries, 0 to 637108
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   fecha           637109 non-null  datetime64[ns]
 1   franja_horaria  637109 non-null  int64         
 2   barrio          637109 non-null  object        
 3   delitos         637109 non-null  object        
 4   id_comuna       637109 non-null  object        
 5   anio            637109 non-null  int64         
 6   mes             637109 non-null  int64         
 7   dia_semana      637109 non-null  object        
 8   horario         637109 non-null  object        
 9   semana          637109 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 53.5+ MB


In [53]:
ts = df.groupby(["fecha"], as_index=False)["delitos"].count().set_index('fecha').dropna()
ts


,delitos
fecha,
2016-01-01,184
2016-01-02,235
2016-01-03,295
2016-01-04,332
2016-01-05,345
...,...
2021-12-27,269
2021-12-28,251
2021-12-29,272


In [55]:
ts['delitos'].rolling(window=10).mean().dropna()

fecha
2016-01-10    298.5
2016-01-11    315.2
2016-01-12    326.3
2016-01-13    331.5
2016-01-14    333.6
              ...  
2021-12-27    249.8
2021-12-28    246.1
2021-12-29    251.4
2021-12-30    245.9
2021-12-31    228.7
Name: delitos, Length: 2183, dtype: float64

In [71]:
fig = px.line()

# Calcular la media móvil de 10 periodos
ts['media_movil'] = ts['delitos'].rolling(window=50).mean()

# Agregar la serie de tiempo con transparencia
fig.add_trace(go.Scatter(x=ts.index, y=ts['delitos'], mode='lines', name='Delitos',
                        line=dict(color='blue'), opacity=0.4))  # Color azul con transparencia

# Agregar la media móvil con un color distinguible
fig.add_trace(go.Scatter(x=ts.index, y=ts['media_movil'], mode='lines', name='Media Móvil (10 períodos)',
                         line=dict(color='red')))  # Color rojo


# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Fecha', yaxis_title='Cantidad de Delitos')
fig.show()

In [28]:
fig = px.line(ts, x='fecha', y="delitos")
fig.show()

In [26]:
import plotly.graph_objects as go

In [19]:
df.groupby(["anio","semana"], as_index=False)["delitos"].count().sort_values(["anio", "semana"],  ascending=True)

,anio,semana,delitos
0,2016,0,714
1,2016,1,2271
2,2016,2,2329
3,2016,3,2266
4,2016,4,2207
...,...,...,...
313,2021,48,1846
314,2021,49,1703
315,2021,50,2047
316,2021,51,1722


In [21]:
px.bar(data_frame=df.groupby(["anio","semana"], as_index=False)["delitos"].count().sort_values(["anio", "semana"],  ascending=True),
       x='semana',
       y='delitos', 
       color= "anio",
       color_discrete_sequence=["red", "green", "blue", "goldenrod", "magenta"],
       facet_col='anio')

In [23]:
px.bar(data_frame=df.loc[df.anio.isin([2019,2020,2021])].groupby(["anio","semana"], 
                                   as_index=False)["delitos"].count().sort_values("delitos", 
                                                                                      ascending=False),
       x='semana',
       y='delitos', 
       color= "anio",
       color_discrete_sequence=["red", "green", "blue", "goldenrod", "magenta"],
       facet_col='anio')